# 署名を作成しよう

- データに署名を行うことでデータの改竄を防ぐことができます。

- ハイブリッド暗号の仕組みを見てみよう
---

### ・セクション1: 関数の準備

In [ ]:
#ライブラリのインストール
%pip install pycryptodome

In [1]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.Hash import SHA256
from Crypto.Signature import pss
import os

In [2]:
#公開鍵の生成
def generate_keys():
    key = RSA.generate(2048)
    private_key = key
    public_key = key.publickey()
    return private_key, public_key

In [3]:
#共通鍵の作成とメッセージの暗号化
def hybrid_encrypt(message, public_key):
    #1.共通鍵（セッションキー）の生成
    session_key = os.urandom(32)
    
    #2.共通鍵を送信相手の公開鍵で暗号化
    cipher_rsa = PKCS1_OAEP.new(public_key, hashAlgo=SHA256)
    encrypted_key = cipher_rsa.encrypt(session_key)
    
    #3.共通鍵でメッセージを暗号化
    cipher_aes = AES.new(session_key, AES.MODE_GCM)
    nonce = cipher_aes.nonce
    
    #4.暗号化と同時に認証タグを生成
    c, tag = cipher_aes.encrypt_and_digest(message)
    
    #暗号化された共通鍵、nonce、署名、暗号文を結合して返す
    return encrypted_key, nonce, tag, c

In [4]:
#復号関数
def hybrid_decrypt(encrypted_key, nonce, tag, c, private_key):
    #1.秘密鍵でセッションキーを復号
    cipher_rsa = PKCS1_OAEP.new(private_key, hashAlgo=SHA256)
    session_key = cipher_rsa.decrypt(encrypted_key)
    
    #2.共通鍵でメッセージを復号
    cipher_aes = AES.new(session_key, AES.MODE_GCM, nonce=nonce)
    
    #3.復号と同時に認証タグの検証を行う
    try:
        m = cipher_aes.decrypt_and_verify(ciphertext, tag)
        return m
    except ValueError:
        raise ValueError("メッセージまたはタグが改ざんされています (認証失敗)")

In [5]:
#署名の作成(送信側)
def sign_message(data, private_key):
    h = SHA256.new(data)
    signer = pss.new(private_key)
    signature = signer.sign(h)
    return signature

#署名の検証(受信側)
def verify_signature(data, signature, public_key):
    h = SHA256.new(data)
    verifier = pss.new(public_key)
    
    try:
        verifier.verify(h, signature)
        return True
    except (ValueError, TypeError):
        return False

---

### ・セクション３: 先生にメッセージを送ってみよう

In [ ]:
#鍵ペアの生成
#教員の署名用
T_private_key, T_public_key = generate_keys()
#生徒の暗号化用
S_private_key, S_public_key = generate_keys()
print("鍵ペアが生成されました。\n")

In [6]:
#鍵ペアの生成
#教員の署名用
T_private_key, T_public_key = generate_keys()
#生徒の暗号化用
S_private_key, S_public_key = generate_keys()
print("鍵ペアが生成されました。\n")

鍵ペアが生成されました。



In [7]:
"""
演習課題(1)
"""
#送信処理

#メッセージ作成
message = input("メッセージを入力: ").encode('utf-8')

#デジタル署名
signature = sign_message(message, S_private_key)
print(f"送信者がメッセージに署名しました。")

#ハイブリッド暗号化
encrypted_session_key, nonce, tag, ciphertext = hybrid_encrypt(message, T_public_key)
print(f"メッセージを暗号化し、共通鍵を受信者の公開鍵で包みました。\n")

メッセージを入力:  あ


送信者がメッセージに署名しました。
メッセージを暗号化し、共通鍵を受信者の公開鍵で包みました。



In [8]:
#先生は以下を押して正しくメッセージが送信されたか確認
try:
    decrypted_message = hybrid_decrypt(encrypted_session_key, nonce, tag, ciphertext, T_private_key)
    print("メッセージが復号されました。")
    
    #デジタル署名の検証
    is_valid = verify_signature(decrypted_message, signature, S_public_key)
    print(f"署名の検証結果: {is_valid}")

    if is_valid:
        print(f"メッセージは教師から送信され、改ざんされていないことが確認できました。")
        print(f"復号されたメッセージ: 「{decrypted_message.decode()}」")
    else:
        print("署名が不正です。送信者が異なるか、メッセージが改ざんされています。")
        
except ValueError as e:
    print(f"復号失敗: {e}")

メッセージが復号されました。
署名の検証結果: True
メッセージは教師から送信され、改ざんされていないことが確認できました。
復号されたメッセージ: 「あ」
